In [134]:
import pymysql
import pandas as pd

host_name = 'localhost'
host_port = 3306
username='root'
password=''
database_name='news_team_4'

db = pymysql.connect(
    host=host_name,
    port=host_port,
    user=username,
    password=password,
    db=database_name,
    charset='utf8'
)

SQL='SELECT * FROM tb_comment'
df=pd.read_sql(SQL, db)

/var/folders/xr/x8smbq7d6l1g33fxnfy1j4b80000gn/T/ipykernel_7953/2980856503.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(SQL, db)


In [ ]:
df.shape

(2504503, 5)

In [135]:
# 댓글 내용이 없는(= 신고당해 삭제된 댓글) 댓글 삭제
df = df[df['Content'] != '']
df.shape

(2504440, 5)

In [136]:
df1=df.copy()
df1.shape

(2504440, 5)

### 강사님 피드백
1. 사용자 별 읽은 기사 목록을 만든다.
2. 사용자 간 읽은 기사 목록을 비교해서 그 행렬로 유사도를 계산한다.
3. 유사도가 높은 유저를 목록으로 or 유사도가 높은 유저가 읽은 뉴스기사 목록을 저장한다.


In [73]:
df1.groupby('UserID').count()

,ID,NewsID,WritedAt,Content
UserID,,,,
19,1,1,1,1
29,1,1,1,1
39,1,1,1,1
40,1,1,1,1
74,1,1,1,1
...,...,...,...,...
1058326,15,15,15,15
1058327,19,19,19,19
1058328,5,5,5,5


In [137]:
# 유저 별 읽은 기사 딕셔너리 생성
dict_df=df1.groupby('UserID')['NewsID'].apply(set).to_dict()

In [88]:
dict_df

{19: {376817},
 29: {376829},
 39: {376826},
 40: {376826},
 74: {376826},
 94: {376826},
 106: {376826},
 111: {376826},
 113: {376829},
 122: {376829},
 131: {376829},
 147: {376829},
 156: {376829},
 178: {376829},
 182: {376829},
 183: {376829},
 185: {376829},
 187: {376829},
 189: {376829},
 197: {376832},
 205: {376830},
 218: {376830},
 221: {376830},
 226: {376830},
 231: {376830},
 248: {376830},
 260: {376830},
 262: {376830},
 268: {376831},
 271: {376831},
 282: {376831},
 283: {376831},
 295: {376831},
 299: {376831},
 300: {376831},
 302: {376831},
 305: {376831},
 313: {376831},
 333: {376833},
 353: {376833},
 354: {376833},
 374: {376832},
 393: {376836},
 396: {376836},
 399: {376836},
 400: {376836},
 402: {376836},
 406: {376836},
 409: {376836},
 418: {376836},
 423: {376836},
 427: {376836},
 432: {376836},
 435: {376836},
 439: {376836},
 442: {376836},
 443: {376836},
 444: {376836},
 445: {376836},
 446: {376836},
 448: {376836},
 453: {376836},
 454: {376836}

In [138]:
# 유저 별 읽은 기사 Set 저장
user_news_df=pd.DataFrame(list(dict_df.items()),
                   columns=['UserID', 'ReadNews'])
user_news_df

,UserID,ReadNews
0,19,{376817}
1,29,{376829}
2,39,{376826}
3,40,{376826}
4,74,{376826}
...,...,...
813963,1058326,"{1227750, 1239015, 1236793, 1165740, 1246190, ..."
813964,1058327,"{1220600, 1220832, 1241667, 1167974, 1242266, ..."
813965,1058328,"{1184720, 1235898, 1246190, 1163143}"
813966,1058329,"{1241219, 1232264, 1234573, 1187856, 1224467, ..."


In [ ]:
# import math
# user_news_df['cos_sim'] =''
# for i in range(813968): # len(user_news_df)+1
#     for j in range(813968):
#         tuple_list = []
#         if i == j:
#             pass
#         cos = len(user_news_df.iloc[i,1] & user_news_df.iloc[j,1])/(math.sqrt(len(user_news_df.iloc[i,1]))*math.sqrt(len(user_news_df.iloc[j,1])))
#         usr_tuple=(user_news_df['UserID'].iloc[j], cos)
#         tuple_list.append(usr_tuple)
#     user_news_df['cos_sim'].iloc[i] = tuple_list

In [154]:
test_df=user_news_df[:1000]
test_df

,UserID,ReadNews,cos_sim
0,19,{376817},"[(29, 0.0), (39, 0.0), (40, 0.0), (74, 0.0), (..."
1,29,{376829},"[(19, 0.0), (39, 0.0), (40, 0.0), (74, 0.0), (..."
2,39,{376826},"[(19, 0.0), (29, 0.0), (40, 1.0), (74, 1.0), (..."
3,40,{376826},"[(19, 0.0), (29, 0.0), (39, 1.0), (74, 1.0), (..."
4,74,{376826},"[(19, 0.0), (29, 0.0), (39, 1.0), (40, 1.0), (..."
...,...,...,...
995,5401,{377256},
996,5403,{377256},
997,5404,{377256},
998,5405,{377256},


In [157]:
import math

test_df['cos_sim'] =''
for i in range(1000): # len(user_news_df)+1
    tuple_list = []
    for j in range(1000):
        if i == j:
            pass
        else:
            # 코사인 유사도 계산식 구현
            cos = len(test_df.iloc[i,1] & test_df.iloc[j,1])/(math.sqrt(len(test_df.iloc[i,1]))*math.sqrt(len(test_df.iloc[j,1])))
            usr_tuple=(test_df['UserID'].iloc[j], cos)
            tuple_list.append(usr_tuple)
    test_df['cos_sim'].iloc[i] = tuple_list

/var/folders/xr/x8smbq7d6l1g33fxnfy1j4b80000gn/T/ipykernel_7953/538517401.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cos_sim'] =''
/var/folders/xr/x8smbq7d6l1g33fxnfy1j4b80000gn/T/ipykernel_7953/538517401.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cos_sim'].iloc[i] = tuple_list


In [158]:
for i in test_df['cos_sim']:
    i.sort(key=lambda x:-x[1])
    
print(test_df['cos_sim'])

,UserID,ReadNews,cos_sim
0,19,{376817},"[(29, 0.0), (39, 0.0), (40, 0.0), (74, 0.0), (..."
1,29,{376829},"[(19, 0.0), (39, 0.0), (40, 0.0), (74, 0.0), (..."
2,39,{376826},"[(19, 0.0), (29, 0.0), (40, 1.0), (74, 1.0), (..."
3,40,{376826},"[(19, 0.0), (29, 0.0), (39, 1.0), (74, 1.0), (..."
4,74,{376826},"[(19, 0.0), (29, 0.0), (39, 1.0), (40, 1.0), (..."
...,...,...,...
995,5401,{377256},"[(19, 0.0), (29, 0.0), (39, 0.0), (40, 0.0), (..."
996,5403,{377256},"[(19, 0.0), (29, 0.0), (39, 0.0), (40, 0.0), (..."
997,5404,{377256},"[(19, 0.0), (29, 0.0), (39, 0.0), (40, 0.0), (..."
998,5405,{377256},"[(19, 0.0), (29, 0.0), (39, 0.0), (40, 0.0), (..."


In [ ]:
test_df['cos_sim']
lambda x : -x[1]

### - 샘플 데이터로 테스트

In [ ]:
# 5만개 데이터로 test
sample_df=df.sample(frac=0.01, random_state=11)
sample_df.shape

(25044, 5)

In [ ]:
sample_df.tail()

,ID,NewsID,UserID,WritedAt,Content
1904069,2101928,169174,894393,2022-11-15 19:09:00,말그대로 개판이네
1489962,1687821,1013437,706382,2022-11-23 06:46:00,"""UN""무용론이 확대될듯."
1444680,1642539,1044078,713391,2022-11-30 16:14:00,가나가 이기면 땡....
279238,279239,391439,359774,0000-00-00 00:00:00,인파가 몰리는 관광지 마다 경찰이 몇백명씩 지키고 있어야 된다는 생각은 어디서 나오...
1092538,1290397,1027247,556832,2022-11-02 11:32:00,음벰바 개1짜2쓱


In [ ]:
print(sample_df['NewsID'].value_counts())
print(sample_df['UserID'].value_counts())

639135     81
1039180    46
1019043    36
1025993    32
1023657    31
           ..
391450      1
452589      1
186666      1
505627      1
1027247     1
Name: NewsID, Length: 17520, dtype: int64
434381    5
432315    5
434596    5
691332    5
434476    4
         ..
723077    1
940222    1
834784    1
919048    1
556832    1
Name: UserID, Length: 23927, dtype: int64


# 협업 필터링
### 1. 최근접 이웃 필터링
- 사용자 기반(User-User) 
    : 당신과 비슷한 고객들이 다음 상품도 구매했습니다.
- 아이템 기반(Item-Item) 
    : 이 상품을 구매한 다른 고객들은 다음 상품도 구매했습니다.

#### 1-1. 아이템 기반 최근접 이웃 협업 필터링
- 이 기사를 본 사람들은 다음 기사도 보았습니다.

In [ ]:
# UserID, NewsID 행렬 생성 
# tab = pd.crosstab(sample_df['UserID'],sample_df['NewsID'])
tab = pd.crosstab(sample_df['NewsID'],sample_df['UserID'])
tab

UserID,1229,2068,2835,3442,3803,4366,4459,4865,5018,6738,...,1058215,1058228,1058248,1058258,1058262,1058278,1058285,1058288,1058292,1058307
NewsID,,,,,,,,,,,,,,,,,,,,,
120512,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120526,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1245909,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1246010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 댓글 단 뉴스기사 간의 유사도를 산출하기 위해 행렬 전치
tab_matrix_T=tab.transpose()
tab_matrix_T.head(3)

NewsID,120512,120516,120521,120522,120526,120527,120534,120546,120549,120561,...,1245507,1245711,1245825,1245837,1245850,1245856,1245909,1246010,1246018,1246164
UserID,,,,,,,,,,,,,,,,,,,,,
1229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 뉴스 기사 간 cosine_similarity 계산
item_sim=cosine_similarity(tab_matrix_T, tab_matrix_T)

item_sim_df=pd.DataFrame(data=item_sim, index=tab.columns, columns=tab.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(23927, 23927)


UserID,1229,2068,2835,3442,3803,4366,4459,4865,5018,6738,...,1058215,1058228,1058248,1058258,1058262,1058278,1058285,1058288,1058292,1058307
UserID,,,,,,,,,,,,,,,,,,,,,
1229,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2835,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
item_sim_df[434596].sort_values(ascending=False)[:15]
# print(sample_df[sample_df['NewsID']=='376826'])

/var/folders/xr/x8smbq7d6l1g33fxnfy1j4b80000gn/T/ipykernel_7953/2677503496.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  item_sim_df[434596].sort_values(ascending=False)[:15]


UserID
434596    1.000000
392095    0.447214
1229      0.000000
815163    0.000000
815532    0.000000
815354    0.000000
815342    0.000000
815293    0.000000
815282    0.000000
815276    0.000000
815246    0.000000
815241    0.000000
815235    0.000000
815154    0.000000
815672    0.000000
Name: 434596, dtype: float64

In [ ]:
# sample_df[sample_df['UserID']==2835] # 377168
# sample_df[sample_df['UserID']==336849] # 377168
sample_df[sample_df['UserID']==392095] # 377168
sample_df[sample_df['NewsID']==527577]
# sample_df[sample_df['NewsID']==655031]
# sample_df[sample_df['NewsID']==618203]
# sample_df[sample_df['NewsID']==617442]
# sample_df[sample_df['NewsID']==611841]
# sample_df[sample_df['NewsID']==629312]
# sample_df[sample_df['NewsID']==629399]

,ID,NewsID,UserID,WritedAt,Content
514156,514157,527577,434596,2022-11-25 13:01:50,"탈중국이 정답...전세계 중국생산거점 기업, 투자금은 탈중화되어 아시아는 한국중심으..."
589744,589745,527577,392095,2022-11-26 08:05:22,탈중국해도 한국에 올 국가는 거이없음\n인건비 비싸고 강성노조에 세금 겁나 많이 내...


### 2. 잠재 요인 협업 필터링

#### 2-1. 행렬 분해를 이용한 잠재 요인 협업 필터링

In [ ]:
pass

### 15.8 파이썬 추천 패키지 - Surprise

In [ ]:
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise.dataset import DatasetAutoFolds

In [ ]:
col='NewsID UserID'
reader = Reader(line_format=col, sep=',', rating_scale=(1,5))
data=sample_df

train_set, test_set = train_test_split(data, test_size=25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정
algo=SVD(n_epochs=20, n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(train_set)


pred = algo.test(test_set)
accuracy.rmse(pred)